In [1]:
%load_ext autoreload
%autoreload 2
import carla
import math
import random

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()
world_map = world.get_map()
spawn_points = world_map.get_spawn_points()
start_point = spawn_points[0]

In [3]:
# Get the blueprint of a black car 
blueprint_library = world.get_blueprint_library()
vehicle_bp = random.choice(blueprint_library.filter('vehicle.toyota.prius'))
vehicle_bp.set_attribute('color', '0,0,0')


In [4]:
# Spawn a vehicle on where I am looking at

# first get where I am looking at
# spectator = world.get_spectator()
# spectator_transform = spectator.get_transform()
# location = spectator_transform.location
# rotation = spectator_transform.rotation





# distance_ahead = 10  # Spawn the car 10 meters in front
# yaw = math.radians(rotation.yaw)  # Convert yaw to radians

# spawn_location = carla.Location(
#     x=location.x + distance_ahead * math.cos(yaw),
#     y=location.y + distance_ahead * math.sin(yaw),
#     z=location.z  # Keep the Z-height same (adjust if needed)
# )

# vehicle = world.try_spawn_actor(vehicle_bp, start_point)h
# spawn_transform = carla.Transform(spawn_location, rotation)
# jake = world.try_spawn_actor(vehicle_bp, spawn_transform)

# Spawn the vehicle
# jake = world.try_spawn_actor(vehicle_bp, spawn_transform)
# Spawn a vehicle on where I am looking at

In [14]:
# spawn jake at the left side of the parking lot 

jake_location = carla.Location(x=-58.468666, y=188.412048, z=0.1)
jake_rotation = carla.Rotation(pitch=0, yaw=-90, roll=0) 

jake_transform = carla.Transform(jake_location, jake_rotation)
# Spawn the camera at the spectator's position and orientation
jake = world.spawn_actor(vehicle_bp, jake_transform)



RuntimeError: Spawn failed because of collision at spawn position

In [6]:
# # Spawn a BEV camera and segment stuff using deeplabv3

# import cv2
# import numpy as np
# import torch
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
# model.eval()
# COLORS = np.array([
#     [255, 0, 0], [0, 255, 0], [0, 0, 255], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
#     [128, 64, 64], [64, 128, 128], [64, 128, 64], [64, 64, 196], [196, 128, 0], [196, 128, 196],
#     # Add more colors as needed for other classes
# ])

# # create the BEV camera above the parking lot 
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '800')
# camera_bp.set_attribute('image_size_y', '600')
# camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed
# camera_bp.set_attribute('sensor_tick', '3.0')
# bev_location = carla.Location(x=-58.686981, y=166.857986, z=24.152607) 
# spectator_rotation = carla.Rotation(pitch=-90, yaw=-180, roll=0) 

# # Create a transform using the given location and rotation
# bev_transform = carla.Transform(bev_location, spectator_rotation)
# # Spawn the camera at the spectator's position and orientation
# camera = world.spawn_actor(camera_bp, bev_transform)

# def process_image(image):
#     array = np.frombuffer(image.raw_data, dtype=np.uint8)
#     array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
#     image_rgb = array[:, :, :3]  # Discard the alpha channel

#     # Preprocess image for DeepLabv3 (resize and normalize)
#     input_image = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
#     input_image = torch.tensor(input_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0

#     # Run the model on the image
#     with torch.no_grad():
#         output = model(input_image)['out'][0]  # Get model's output
#     output_predictions = output.argmax(0)  # Get the class predictions (highest logit)
    
#     # Convert the predictions to colors
#     seg_image = COLORS[output_predictions.numpy()]
    
#     # Create the masked image by blending the segmentation output with the original image
#     masked_image = np.copy(image_rgb)
#     masked_image[output_predictions.numpy() == 0] = [0, 0, 0]  # Optionally mask out background (class 0)

#     # Overlay the segmentation result on the original image
#     blended_image = cv2.addWeighted(image_rgb, 0.7, seg_image.astype(np.uint8), 0.3, 0)
#     unique_values = np.unique(output_predictions)
#     # Display the result
#     cv2.imshow("BEV + Segmentation", blended_image)
#     cv2.waitKey(1)
    


In [7]:
# # Spawn a camera in front of jake and do segmenation using YOLO
# import cv2
# import numpy as np
# import threading
# # from yolo import *
# from ultralytics import YOLO
# seg_model = YOLO("yolov8n.pt") 


# # create the BEV camera above the parking lot 
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '800')
# camera_bp.set_attribute('image_size_y', '600')
# camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed
# # camera_bp.set_attribute('sensor_tick', '5.0')
# camera_transform = carla.Transform(carla.Location(x=0, y=5, z=2))  # Adjust if needed

# camera = world.spawn_actor(camera_bp, camera_transform, attach_to=jake)

# def process_image(image):
#     array = np.frombuffer(image.raw_data, dtype=np.uint8)
#     array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
#     array = array[:, :, :3]  # Remove alpha channel
#     results = seg_model(array)  # Perform inference
#     annotated_frame = results[0].plot()  # Get the annotated image
#     cv2.imshow("BEV + Segmentation", annotated_frame)
    
#     # Check if window is closed
#     # key = cv2.waitKey(1) & 0xFF
#     # if cv2.getWindowProperty("BEV + Segmentation", cv2.WND_PROP_VISIBLE) < 1:
#     #     running = False
#     #     camera.stop()
#     #     cv2.destroyAllWindows()

#     cv2.waitKey(1)
# camera.listen(lambda image: process_image(image))

In [5]:
# Spawn a camera in front of jake and do segmenation using DeepLabv3
import cv2
import numpy as np
import torch
import threading 
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()
COLORS = np.array([
    [255, 0, 0], [0, 255, 0], [0, 0, 255], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
    [128, 64, 64], [64, 128, 128], [64, 128, 64], [64, 64, 196], [196, 128, 0], [196, 128, 196],
    # Add more colors as needed for other classes
])



# create the BEV camera above the parking lot 
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '800')
camera_bp.set_attribute('image_size_y', '600')
camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed
# camera_bp.set_attribute('sensor_tick', '1.0')
camera_bp.set_attribute('sensor_tick', '0.033333')  # 30 FPS 
# camera_bp.set_attribute('sensor_tick', '5.0')
camera_transform = carla.Transform(carla.Location(x=0, y=0, z=2))  # Adjust if needed

camera = world.spawn_actor(camera_bp, camera_transform, attach_to=jake)

def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
    image_rgb = array[:, :, :3]  # Discard the alpha channel

    # Preprocess image for DeepLabv3 (resize and normalize)
    input_image = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
    input_image = torch.tensor(input_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0

    # Run the model on the image
    with torch.no_grad():
        output = model(input_image)['out'][0]  # Get model's output
    output_predictions = output.argmax(0)  # Get the class predictions (highest logit)
    
    # Convert the predictions to colors
    seg_image = COLORS[output_predictions.numpy()]
    
    # Create the masked image by blending the segmentation output with the original image
    masked_image = np.copy(image_rgb)
    masked_image[output_predictions.numpy() == 0] = [0, 0, 0]  # Optionally mask out background (class 0)

    # Overlay the segmentation result on the original image
    blended_image = cv2.addWeighted(image_rgb, 0.7, seg_image.astype(np.uint8), 0.3, 0)
    # unique_values = np.unique(output_predictions)
    # Display the result
    display_queue.put(blended_image)
    # cv2.imshow("POV + Segmentation", blended_image)

    # cv2.waitKey(1)


# normal display without segmentation 
def process_image(image):
    # Convert the image to a numpy array
    image_array = np.array(image.raw_data)
    image_array = image_array.reshape((image.height, image.width, 4))  # Change to a 3D array (height, width, RGBA)

    # Convert to BGR (OpenCV format)
    image_bgr = image_array[:, :, :3]

    # Use a queue to safely pass images to the display thread
    display_queue.put(image_bgr)

def display_image():
    while True:
        # Check if the queue has a new image to display
        if not display_queue.empty():
            image_bgr = display_queue.get()
            cv2.imshow('RGB Cam + Segmentation', image_bgr)
            cv2.waitKey(1)

# Start the display thread
import queue
display_queue = queue.Queue()

display_thread = threading.Thread(target=display_image)
display_thread.daemon = True
display_thread.start()

camera.listen(lambda image: threading.Thread(target=process_image, args=(image,)).start())


Using cache found in N:\windat.v2/.cache\torch\hub\pytorch_vision_v0.10.0


In [6]:
jake.set_autopilot(True)

In [6]:

import glob
import os
import sys
import argparse
import time
from datetime import datetime
import random
import threading
import numpy as np
from mayavi import mlab


def lidar_callback(point_cloud, buf):
    """Prepares a point cloud with intensity colors and stores in a buffer, which updated the mayavi visualisation"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))

    # Isolate the 3D data
    points = data[:, :-1]

    # We're negating the y to correclty visualize a world that matches
    # what we see in Unreal since Mayavi uses a right-handed coordinate system
    points[:, :1] = -points[:, :1]

    # # An example of converting points from sensor to vehicle space if we had
    # # a carla.Transform variable named "tran":
    # points = np.append(points, np.ones((points.shape[0], 1)), axis=1)
    # points = np.dot(tran.get_matrix(), points.T).T
    # points = points[:, :-1]

    #copy points/intensities into buffer
    buf['pts'] = points
    buf['intensity'] = intensity

def semantic_lidar_callback(point_cloud, buf):
    """Prepares a point cloud with semantic segmentation colors"""
    data = np.frombuffer(point_cloud.raw_data, dtype=np.dtype([
        ('x', np.float32), ('y', np.float32), ('z', np.float32),
        ('CosAngle', np.float32), ('ObjIdx', np.uint32), ('ObjTag', np.uint32)]))

    # We're negating the y to correclty visualize a world that matches
    # what we see in Unreal since Open3D uses a right-handed coordinate system
    points = np.array([data['x'], -data['y'], data['z']]).T

    # # An example of adding some noise to our data if needed:
    # points += np.random.uniform(-0.05, 0.05, size=points.shape)

    # Colorize the pointcloud based on the CityScapes color palette
    labels = np.array(data['ObjTag'], dtype=np.float32)
    int_color = labels

    # # In case you want to make the color intensity depending
    # # of the incident ray angle, you can use:
    #  int_color *= np.array(data['CosAngle'])

    buf['pts'] = points
    buf['intensity'] = labels


def carlaEventLoop(world):
        frame = 0
        dt0 = datetime.now()
        while True:
            time.sleep(0.005)
            world.tick()

            process_time = datetime.now() - dt0
            sys.stdout.write('\r' + 'FPS: ' + str(1.0 / process_time.total_seconds()))
            sys.stdout.flush()
            dt0 = datetime.now()
            frame += 1
try:
    original_settings = world.get_settings()
    settings = world.get_settings()
    traffic_manager = client.get_trafficmanager(8000)
    traffic_manager.set_synchronous_mode(True)

    delta = 0.05

    settings.fixed_delta_seconds = delta
    settings.synchronous_mode = True
    world.apply_settings(settings)
    # spawn jake at the left side of the parking lot 

    jake_location = carla.Location(x=-58.468666, y=188.412048, z=7.642653)
    jake_rotation = carla.Rotation(pitch=0, yaw=-90, roll=0) 

    jake_transform = carla.Transform(jake_location, jake_rotation)
    # Spawn the camera at the spectator's position and orientation
    jake = world.spawn_actor(vehicle_bp, jake_transform)


    jake.set_autopilot(True)

    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
    # lidar_bp.set_attribute('sensor_tick', '5.0')
    lidar_transform = carla.Transform(carla.Location(x=0, y=5, z=0))  # Adjust if needed

    lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=jake)

    fig = mlab.figure(size=(960,540), bgcolor=(0.05,0.05,0.05))
    vis = mlab.points3d(0, 0, 0, 0, mode='point', figure=fig)
    mlab.view(distance=25)
    buf = {'pts': np.zeros((1,3)), 'intensity':np.zeros(1)}

    #  @mlab.animate(delay=100)
    def anim():
        i=0
        while True:
            vis.mlab_source.reset(x=buf['pts'][:,0], y=buf['pts'][:,1], z=buf['pts'][:,2], scalars=buf['intensity'])
            mlab.savefig(f'{i}.png', figure=fig)
            time.sleep(0.1)
            i+=1

    # lidar.listen(lambda data: semantic_lidar_callback(data, buf))
    lidar.listen(lambda data: lidar_callback(data, buf))

    loopThread = threading.Thread(target=carlaEventLoop, args=[world], daemon=True)
    loopThread.start()
    anim()
    #  mlab.show()

finally:
    world.apply_settings(original_settings)
    traffic_manager.set_synchronous_mode(False)

    jake.destroy()
    lidar.destroy()

TypeError: unhashable type: 'PolyData'

: 

In [ ]:
# example code to move the car (jake)

control = carla.VehicleControl()
control.throttle = 0.5  # Forward speed
control.steer = -0.5  # Steering angle to move in the desired direction

# Apply the control
jake.apply_control(control)



: 

In [ ]:

# 
control.throttle = 0  # No forward speed
control.steer = 0  # Straight steering to prevent turning

# Apply the control to stop the vehicle
jake.apply_control(control)

In [ ]:

control.throttle = 0.5  # No forward speed
control.steer = 0  # Straight steering to prevent turning
control.reverse= True
# Apply the control to stop the vehicle
jake.apply_control(control)

In [3]:
jake.destroy()

NameError: name 'jake' is not defined

In [8]:
from draw_line import *

jake_location = carla.Location(x=-58.468666, y=188.412048, z=7.642653)

draw_line(world, (-40.468666, 193.412048, 1), (-40.468666, 153.412048, 1))
draw_line(world, (-40.468666, 153.412048, 1), (-70.468666, 153.412048, 1))
draw_line(world, (-70.468666, 193.412048, 1), (-70.468666, 153.412048, 1))
draw_line(world, (-40.468666, 193.412048, 1), (-70.468666, 193.412048, 1))


In [22]:
import time 
from car_control_helper import * 

move_car(jake, steer=-1)
time.sleep(1)
move_car(jake, steer=0)
time.sleep(1)
stop_car(jake)
# move_car(jake, reverse=True, steer = -1)
# time.sleep(1)
# stop_car(jake)

In [26]:
actors = world.get_actors()
list(actors)

In [23]:
import numpy as np
from draw_line import *
start_x = -70.5
end_x = -40.45
start_y =  153.412048
end_y = 193.412048
x_range = np.linspace(start_x, end_x, 100) 
# y_range = np.linspace(start_y, end_y, steps) # can potentially do two orientations to incraese accuracy
for x in x_range:
    start = carla.Vector3D(x, start_y, 1)
    end = carla.Vector3D(x, end_y, 1)
    draw_line(world, (x, start_y, 1), (x, end_y, 1))
    res = world.cast_ray(start, end)    
    for point in res:
        print(point.location, point.label)


Location(x=-70.500000, y=161.683167, z=1.000000) Buildings
Location(x=-70.196465, y=179.692383, z=1.000000) Car
Location(x=-70.196465, y=182.872711, z=1.000000) Car
Location(x=-69.892929, y=174.433441, z=1.000000) Car
Location(x=-69.892929, y=179.388870, z=1.000000) Car
Location(x=-69.892929, y=182.581421, z=1.000000) Car
Location(x=-69.589394, y=170.134445, z=1.000000) Car
Location(x=-69.589394, y=174.292847, z=1.000000) Car
Location(x=-69.589394, y=179.224640, z=1.000000) Car
Location(x=-69.589394, y=182.581421, z=1.000000) Car
Location(x=-69.285858, y=169.612122, z=1.000000) Car
Location(x=-69.285858, y=174.229462, z=1.000000) Car
Location(x=-69.285858, y=179.224640, z=1.000000) Car
Location(x=-69.285858, y=182.581421, z=1.000000) Car
Location(x=-68.982323, y=169.511154, z=1.000000) Car
Location(x=-68.982323, y=174.231201, z=1.000000) Car
Location(x=-68.982323, y=179.224640, z=1.000000) Car
Location(x=-68.982323, y=182.581421, z=1.000000) Car
Location(x=-68.678787, y=169.488708, z=1

In [25]:
# test collision detector 
from hybrid_astar_carla import detect_obstacles

start_x = -70.5
end_x = -40.45
start_y =  153.412048
end_y = 193.412048
print(detect_obstacles(world, start_x, end_x, start_y, end_y))

{(-50.4666633605957, 157.8984375), (-50.16312789916992, 176.05157470703125), (-65.94696807861328, 169.64183044433594), (-57.75151062011719, 186.48297119140625), (-57.75151062011719, 186.9343719482422), (-51.07373809814453, 165.450439453125), (-49.25252151489258, 165.450439453125), (-54.71615982055664, 186.4828643798828), (-68.37525177001953, 174.2356414794922), (-52.59141540527344, 176.05157470703125), (-66.85757446289062, 179.22463989257812), (-65.6434326171875, 182.5814208984375), (-55.3232307434082, 186.48287963867188), (-55.3232307434082, 186.76248168945312), (-52.89495086669922, 165.450439453125), (-68.07171630859375, 169.4480438232422), (-66.55403900146484, 174.24891662597656), (-45.6100959777832, 186.48394775390625), (-45.6100959777832, 186.7643585205078), (-51.37727355957031, 157.8917999267578), (-67.46464538574219, 169.45852661132812), (-70.19646453857422, 179.6923828125), (-49.25252151489258, 158.05514526367188), (-48.64545440673828, 165.80953979492188), (-50.77020263671875, 

: 